# Custom conv

#####  LENET uczony na 2000 * 50 samplach

In [1]:
#Wraper podstawiajacy 
def py_func(func, inp, Tout, stateful=True, name=None):
    
    # Nowa nazwa, żeby móc modyfikować nasz gradient
    rnd_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+8))
    tf.RegisterGradient(rnd_name)(_BinGrad)
    
    #Podstawienie gradientu
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": rnd_name}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)

#Wrapper na wrappera
def custom_conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name='MySquare'):
    return py_func(conv, [x,W], [tf.float32])[0]
    
#Skopiowany gradient z conv2d
def _BinGrad(op, grad):
    return [nn_ops.conv2d_backprop_input(
    array_ops.shape(op.inputs[0]), op.inputs[1], grad,(1,1,1,1),
    'SAME', False, 'NHWC'),
      nn_ops.conv2d_backprop_filter(op.inputs[0],
                                    array_ops.shape(op.inputs[1]), grad,
                                    (1,1,1,1),
                                    'SAME',
                                    False,
                                    'NHWC')]

#Nasza operacja
#result[batch,0:conv_height,0:conv_width,w_filter] -> suma konwolucji wszystkich channeli dla konkretnego batcha
#i filtra
def conv(x, W, strides=[1, 1, 1, 1], padding='SAME'):
    batches,x_height,x_width,x_channels = x.shape
    W_height,W_width,w_channels,w_filters = W.shape
    assert w_channels == x_channels
    result = None
    for batch in range(batches):
        for w_filter in range(w_filters):
            for channel in range(w_channels):
                inp = x[batch,0:x_height,0:x_width,channel]
                fil = np.rot90(W[0:W_height,0:W_width,channel,w_filter], 2)
                n = W_height*W_width
                a = np.sum(np.abs(fil)) / n
                conv_result = signal.convolve2d(inp, np.sign(fil), 'same') * a
                conv_height,conv_width = conv_result.shape
                if result is None:
                    result = np.zeros((batches,conv_height,conv_width,w_filters), dtype=np.float32)
                tmp = result[batch,0:conv_height,0:conv_width,w_filter]
                result[batch,0:conv_height,0:conv_width,w_filter] = np.add(tmp, conv_result)
    return result

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import time

from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import function
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import sparse_ops
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
from scipy import signal
from tensorflow.python.ops import nn_ops, array_ops

FLAGS = None

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# Import data
mnist = input_data.read_data_sets('/tmp/dataset', one_hot=True)

# Create the model
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = weight_variable([5, 5, 1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(custom_conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5,32, 32])
b_conv2 = bias_variable([32])
h_conv2 = tf.nn.relu(custom_conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 32, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

sess = tf.Session()
sess.as_default()

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

time_start = time.time()

for i in range(4000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(session=sess, feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    #print(W_conv2.eval(session = sess)[0][0][0])
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(session=sess, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print(time.time() - time_start)

print("test accuracy %g"%accuracy.eval(session=sess,feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting /tmp/dataset\train-images-idx3-ubyte.gz
Extracting /tmp/dataset\train-labels-idx1-ubyte.gz
Extracting /tmp/dataset\t10k-images-idx3-ubyte.gz
Extracting /tmp/dataset\t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.14
step 100, training accuracy 0.82
step 200, training accuracy 0.94


KeyboardInterrupt: 

In [ ]:
4000 * 50 samples = test accuracy 0.9296